In [18]:
from torchvision import datasets
from torchvision import transforms as T
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnext50_32x4d
from sklearn.metrics import accuracy_score
import torch
import os
from tqdm.notebook import tqdm
from torch import nn, optim 
import math
import imgaug.augmenters as iaa
from random import randint, sample

from PIL.Image import fromarray
import cv2
from scipy.spatial.distance import cosine
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join
from torch import nn
import numpy as np
import json

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


pwd = os.getcwd()

In [2]:
class AdaCos(nn.Module):
    def __init__(self, feat_dim, num_classes, fixed_scale=False):
        super(AdaCos, self).__init__()
        self.fixed_scale = fixed_scale
        self.scale = math.sqrt(2) * math.log(num_classes - 1)
        self.W = nn.Parameter(torch.FloatTensor(num_classes, feat_dim))
        nn.init.xavier_uniform_(self.W)
        
    def forward(self, feats, labels):
        W = F.normalize(self.W)

        logits = F.linear(feats, W)

        theta = torch.acos(torch.clamp(logits, -1.0 + 1e-7, 1.0 - 1e-7))
        one_hot = torch.zeros_like(logits)
        one_hot.scatter_(1, labels.view(-1, 1).long(), 1)

        if self.fixed_scale:
            with torch.no_grad():
                B_avg = torch.where(one_hot < 1, torch.exp(self.scale * logits), torch.zeros_like(logits))
                B_avg = torch.sum(B_avg) / feats.size(0)
                
                theta_med = torch.median(theta[one_hot == 1])
                self.scale = torch.log(B_avg) / torch.cos(torch.min(math.pi/4 * torch.ones_like(theta_med), theta_med))
            
        output = self.scale * logits
        return output
    
    def get_logits(self, feats):
        W = F.normalize(self.W)

        logits = F.linear(feats, W)
        return logits


In [3]:
class ArcFace(nn.Module):
     def __init__(self, feat_dim, num_class, margin_arc=0.5, margin_am=0.0, scale=30):
         super(ArcFace, self).__init__()
         self.weight = nn.Parameter(torch.Tensor(feat_dim, num_class))
         self.weight.data.uniform_(-1, 1).renorm_(2, 1, 1e-5).mul_(1e5)
         self.margin_arc = margin_arc
         self.margin_am = margin_am
         self.scale = scale
         self.cos_margin = math.cos(margin_arc)
         self.sin_margin = math.sin(margin_arc)
         self.min_cos_theta = math.cos(math.pi - margin_arc)

     def forward(self, feats, labels):
         kernel_norm = F.normalize(self.weight, dim=0)
         feats = F.normalize(feats)
         cos_theta = torch.mm(feats, kernel_norm) 
         cos_theta = cos_theta.clamp(-1, 1)
         sin_theta = torch.sqrt(1.0 - torch.pow(cos_theta, 2))
         cos_theta_m = cos_theta * self.cos_margin - sin_theta * self.sin_margin
         
         cos_theta_m = torch.where(cos_theta > self.min_cos_theta, cos_theta_m, cos_theta-self.margin_am)
         index = torch.zeros_like(cos_theta)

         index.scatter_(1, labels.data.view(-1, 1), 1)
         index = index.type(torch.bool)
         output = cos_theta * 1.0
         output[index] = cos_theta_m[index]
         output *= self.scale


         return output

In [4]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM,self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'

In [5]:
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
                
        self.backbone = torch.nn.Sequential(*(list(resnext50_32x4d(pretrained=True).children())[:-2]))
        self.gem_pool = GeM()
        self.bn1 = nn.BatchNorm1d(2048)
        self.fc1 = nn.Linear(2048, 512)
        self.dropout = nn.Dropout(0.2)

        self.arc_face = AdaCos(512, num_classes)
        
    def forward(self, x, targets = None):
        x = torch.squeeze(torch.squeeze(self.gem_pool(self.backbone(x)), -1), -1)

        x = F.relu(self.fc1(self.dropout(self.bn1(x))))
        x = F.normalize(x)
        
        if targets is not None:
            logits = self.arc_face(x, targets)
            return logits

        return x
    
    def get_logits(self, x):
        x = self.gem_pool(self.backbone(x))
        x = torch.unsqueeze(torch.squeeze(x), 0)
        x = F.relu(self.fc1(self.dropout(self.bn1(x))))
        x = F.normalize(x)

        logits = self.arc_face.get_logits(x)
        return logits
        
input_size = (256, 256)

In [6]:
class Trainer():
    
    def __init__(self, criterion = None, optimizer = None, device = None, start_epoch=0):
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = device
        self.start_epoch = start_epoch
        
        
    def accuracy(self, logits, labels):
        ps = torch.argmax(logits,dim = 1).detach().cpu().numpy()
        acc = accuracy_score(ps,labels.detach().cpu().numpy())
        return acc

        
    def train_batch_loop(self, model, train_loader, i, save_path=None):
        
        epoch_loss = 0.0
        epoch_acc = 0.0
        pbar_train = tqdm(train_loader, desc="Epoch" + " [TRAIN] " + str(i+1))
        batch_num = len(pbar_train)
        for it, data in enumerate(pbar_train):
            
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            
            logits = model(images, labels)
            loss = self.criterion(logits,labels)
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            epoch_loss += loss.item()
            epoch_acc += self.accuracy(logits, labels)
            
            postfix = {'loss' : round(float(epoch_loss/(it+1)), 4), 'acc' : float(epoch_acc/(it+1))}
            pbar_train.set_postfix(postfix)
            
            if save_path is not None:
                if it % 200 == 199:
                    with open(save_path + 'train_log.txt', 'a') as f:
                        f.write(f'B# {it+1}/{batch_num}, Loss: {round(float(epoch_loss/(it+1)), 4)}, Acc: {round(float(epoch_acc/(it+1)), 4)} \n')
                
                if it % 2000 == 999:
                    torch.save(model, save_path + 'model_.pth')
                
            
        return epoch_loss / len(train_loader), epoch_acc / len(train_loader)
            
    
    def valid_batch_loop(self, model, valid_loader, i, save_path=None):
        
        epoch_loss = 0.0
        epoch_acc = 0.0
        pbar_valid = tqdm(valid_loader, desc = "Epoch" + " [VALID] " + str(i+1))
        batch_num = len(pbar_valid)
        
        for it, data in enumerate(pbar_valid):
            
            images,labels = data
            images = images.to(device)
            labels = labels.to(device)
            
            logits = model(images, labels)
            loss = self.criterion(logits, labels)
            
            epoch_loss += loss.item()
            epoch_acc += self.accuracy(logits, labels)
            
            postfix = {'loss' : round(float(epoch_loss/(it+1)), 4), 'acc' : float(epoch_acc/(it+1))}
            pbar_valid.set_postfix(postfix)
            
            
            if save_path is not None:
                if it % 200 == 199:
                    with open(save_path + 'valid_log.txt', 'a') as f:
                        f.write(f'B# {it+1}/{batch_num}, Loss: {round(float(epoch_loss/(it+1)), 4)}, Acc: {round(float(epoch_acc/(it+1)), 4)} \n')
            
        return epoch_loss / len(valid_loader), epoch_acc / len(valid_loader)
            
    
    def run(self, model, train_loader, valid_loader=None, schedule=None, epochs=1, save_path=None):
        if not os.path.exists(save_path) and save_path is not None:
            os.mkdir(save_path)
        
        if schedule is not None:
            if len(schedule) != epochs:
                raise Exception('Scedule lenght must be equal epoch num')
        
        
        for i in range(self.start_epoch, self.start_epoch + epochs, 1):
            if save_path is not None:
                epoch_save_path = join(save_path, f'epoch_{i}/')
                if not os.path.exists(epoch_save_path):
                    os.mkdir(epoch_save_path)
            else:
                epoch_save_path = None
            
            if schedule is not None:
                for g in self.optimizer.param_groups:
                    g['lr'] = schedule[i]
            
            model.train()
            avg_train_loss, avg_train_acc = self.train_batch_loop(model, train_loader, i, save_path=epoch_save_path)
            
            if save_path is not None:
                torch.save(model, epoch_save_path + 'model.pth')
            
            if valid_loader is not None:
                model.eval()
                avg_valid_loss, avg_valid_acc = self.valid_batch_loop(model, valid_loader, i, save_path=epoch_save_path)
            
        return model
    
    def run_eval(self, model, data_lodaer):
        model.eval()
        avg_valid_loss, avg_valid_acc = self.valid_batch_loop(model, data_lodaer, 0)
        return avg_valid_loss, avg_valid_acc

In [7]:
class ImageDataset(Dataset):
  def __init__(self, csv, img_folder, transform=None):
    self.transform = transform
    self.img_folder = img_folder
     
    self.images = csv['image']
    self.targets = csv['Y']
   

  def __len__(self):
    return len(self.images)
 

  def __getitem__(self, index):

    image = cv2.cvtColor(cv2.imread(join(self.img_folder, self.images[index])), cv2.COLOR_BGR2RGB)
    target = self.targets[index]
     
    if self.transform is not None:
        image = self.transform(image)
    
    return image, target

In [8]:
csv_path = join(pwd, 'csv/train.csv')
img_data = join(pwd, '../train_images-256-256')

In [9]:
data_csv = pd.read_csv(csv_path)

transforms_list = T.Compose([             
    iaa.Sequential([
        iaa.Sequential([
        iaa.Sometimes(0.3, iaa.AverageBlur(k=(3,3))),
        iaa.Sometimes(0.3, iaa.MotionBlur(k=(3,5))),
        iaa.Add((-10, 10), per_channel=0.5),
        iaa.Multiply((0.9, 1.1), per_channel=0.5),
        iaa.Sometimes(0.3, iaa.Affine(
            scale={'x': (0.9,1.1), 'y': (0.9,1.1)},
            translate_percent={'x': (-0.05,0.05), 'y': (-0.05,0.05)},
            shear=(-10,10),
            rotate=(-10,10)
            )),
        iaa.Sometimes(0.3, iaa.Grayscale(alpha=(0.8,1.0))),
        ], random_order=True),
        iaa.size.Resize(input_size, interpolation='cubic')
    ]).augment_image,     
    T.ToTensor()
])

train_dataset = ImageDataset(data_csv,
                             img_data,
                             transform=transforms_list)

In [10]:
# x = train_dataset[7]
# x = x[0].permute(1, 2, 0).numpy()
# x = (x * 255.).astype('uint8')
# fromarray(x)

In [11]:
batch_size = 48
start_epoch = 0
num_epochs = 20
lr = 0.0001
# schedule = [0.001, 0.00075, 0.0005]
num_classes = data_csv['individual_id'].nunique()
save_path = join(pwd, '../models/renet_50')
lr_start   = 0.000001
lr_max     = 0.000005 * batch_size
lr_min     = 0.000001
lr_ramp_ep = 4
lr_sus_ep  = 0
lr_decay   = 0.9


train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)

In [12]:
def lrfn(epoch):
    if start_epoch != 0:
        epoch = epoch + start_epoch
    if epoch < lr_ramp_ep:
        lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        
    elif epoch < lr_ramp_ep + lr_sus_ep:
        lr = lr_max
        
    else:
        lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
        
    return lr

In [23]:
schedule = [lrfn(i) for i in range(num_epochs)]

In [24]:
model = Net(num_classes=num_classes).to(device)
# model = torch.load('/content/models//model.pth')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

trainer = Trainer(criterion=criterion,
                  optimizer=optimizer,
                  device=device,
                  start_epoch=start_epoch)

In [ ]:
trainer.run(model, train_loader, epochs=num_epochs, save_path=save_path, schedule=schedule)

In [15]:
model = torch.load('/content/resnet_50_ada_cos/epoch_19/model.pth').to(device)
model.eval();

In [15]:
# submsission = []
# img_path = '/content/test_images-256-256/'

# model.eval();


# for img_n in tqdm(os.listdir(img_path)):
#     img = cv2.cvtColor(cv2.imread(img_path + img_n), cv2.COLOR_BGR2RGB)
#     input = torch.unsqueeze(transforms_list(img), 0).to(device)
    
#     logits = model.get_logits(input).detach().cpu().numpy()
#     predict_individuials = np.argsort(logits[0])[::-1][:5]

#     predictions = ' '.join([individial_mapping[i] for i in predict_individuials])


#     submsission.append({'image': img_n, 'predictions': predictions})

  0%|          | 0/27956 [00:00<?, ?it/s]

In [24]:
transforms_list_eval = T.Compose([             
    iaa.Sequential([
        iaa.size.Resize(input_size, interpolation='cubic')
    ]).augment_image,     
    T.ToTensor()
])

def get_embedding(img_path):
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    input = torch.unsqueeze(transforms_list_eval(img), 0).to(device)
    embed = model(input).detach().cpu().numpy()

    return embed

In [25]:
embed = get_embedding('/content/test_images-256-256/0a1eaeebba9d13.jpg')

In [36]:
train_folder = '/content/train_images-256-256/'

img_to_target = json.loads(open('/content/happywhale/data/img_to_target.json', 'r').read())

train_targets = []
train_embeddings = []
for filename in tqdm(os.listdir(train_folder)):
    embeddings = get_embedding(join(train_folder, filename))
    targets = img_to_target[filename]
    train_embeddings.append(embeddings)
    train_targets.append(targets)

  0%|          | 0/51033 [00:00<?, ?it/s]